In [1]:
import os
os.chdir("/content/drive/MyDrive")

In [2]:
import numpy as np
specificity = list(np.load("Preprocessed Data/specificity.npy"))
clusters = list(np.load("TCR-Only Data/clusters.npy"))
z_mean = list(np.load("TCR-Only Data/z_mean.npy"))

In [3]:
from scipy.spatial.distance import pdist, squareform

# data是聚类后的数据
# labels是聚类后每个数据点所属的类别
data = z_mean
labels = clusters

distances = pdist(data)
dist_matrix = squareform(distances)

# 计算每个类别内部数据点的平均距离
cohesions = []
for label in set(labels):
    indices = [i for i, x in enumerate(labels) if x == label]
    dist_sum = sum([dist_matrix[i][j] for i in indices for j in indices])
    cohesion = dist_sum / (len(indices) * (len(indices) - 1) / 2)
    cohesions.append(cohesion)

In [4]:
print(set(labels))

{0, 1, 2, 3, 4}


In [5]:
mapping = ['A0301_KLGGALQAK_IE-1_CMV_binder', 'A0201_ELAGIGILTV_MART-1_Cancer_binder', 'A0201_GILGFVFTL_Flu-MP_Influenza_binder', 'A1101_IVTDFSVIK_EBNA-3B_EBV_binder', 'A1101_AVFDRKSDAK_EBNA-3B_EBV_binder']
print(mapping)

['A0301_KLGGALQAK_IE-1_CMV_binder', 'A0201_ELAGIGILTV_MART-1_Cancer_binder', 'A0201_GILGFVFTL_Flu-MP_Influenza_binder', 'A1101_IVTDFSVIK_EBNA-3B_EBV_binder', 'A1101_AVFDRKSDAK_EBNA-3B_EBV_binder']


In [6]:
print(cohesions)

[1.3902492688966388, 1.7839336470398683, 1.4704287351944938, 0.05352753598694983, 1.4642741636561891]


In [7]:
data = np.array(data).reshape(-1, 2)
print(data)

[[-0.32749015  0.73723215]
 [ 0.82411015 -1.7565324 ]
 [ 1.8046141  -0.96817565]
 ...
 [-0.5593172  -1.2454222 ]
 [-1.4473503  -1.8280607 ]
 [ 0.2300427  -0.59644675]]


In [8]:
# from scipy.spatial.distance import cdist

# # data是聚类后的数据
# # labels是聚类后每个数据点所属的类别

# separations = []
# for label1 in set(labels):
#     indices1 = [i for i, x in enumerate(labels) if x == label1]
#     for label2 in set(labels):
#         if label2 == label1:
#             continue
#         indices2 = [i for i, x in enumerate(labels) if x == label2]
#         dist_sum = sum([cdist(data[[i],:], data[[j],:]) for i in indices1 for j in indices2])
#         separation = dist_sum / (len(indices1) * len(indices2))
#         separations.append(separation)


In [9]:
# print(separations)

In [10]:
# sep = [2.28412107, 1.73917562, 1.68215179, 1.28932635, 2.28412107, 3.60495411, 3.40216338, 1.93118661, 1.73917562, 3.60495411, 2.48272593, 1.93322555, 1.68215179, 3.40216338, 2.48272593, 2.77081828, 1.28932635, 1.93118661, 1.93322555, 2.77081828]

In [11]:
# print(len(sep))
# sep

In [12]:
from sklearn import metrics
import copy
true_labels = specificity
labels_copy = copy.deepcopy(labels)
for i in range(len(labels_copy)):
  labels_copy[i] = mapping[labels_copy[i]]

In [13]:
f_measure_1 = metrics.f1_score(true_labels, labels_copy, average=None)
f_measure_2 = metrics.f1_score(true_labels, labels_copy, average='weighted')
print(f_measure_1)
print(f_measure_2)

[0.00930233 0.83048703 0.59163615 0.13188976 0.90738474]
0.700596257978675


In [15]:
print(np.mean(cohesions))
# print(np.mean(sep))

1.232482670154828


In [16]:
precision_list = []
recall_list = []
accuracy_list = []
iou_list = []
f1_list = []
for cluster in sorted(mapping):
  print(cluster)
  y_test = specificity
  y_pred = copy.deepcopy(clusters)
  for i in range(len(y_pred)):
    y_pred[i] = mapping[y_pred[i]]
  tn, fp, fn, tp = 0, 0, 0, 0
  for i in range(len(y_test)):
    if y_pred[i] == cluster and y_test[i] == cluster:
      tp += 1
    if y_pred[i] == cluster and y_test[i] != cluster:
      fp += 1
    if y_pred[i] != cluster and y_test[i] == cluster:
      fn += 1
    if y_pred[i] != cluster and y_test[i] != cluster:
      tn += 1

  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  accuracy = (tp + tn) / ((tp + fn) + (fp + tn))
  iou = tp / (tp + fn + fp)
  f1 = (2 * tp) / (2 * tp + fn + fp)

  precision_list.append(precision)
  recall_list.append(recall)
  accuracy_list.append(accuracy)
  iou_list.append(iou)
  f1_list.append(f1)

  print("TP: {}".format(tp))
  print("TN: {}".format(tn))
  print("FP: {}".format(fp))
  print("FN: {}".format(fn))
  print()

  print("Precision: {:.4f}".format(precision))
  print("Recall: {:.4f}".format(recall))
  print("Accuracy: {:.4f}".format(accuracy))
  print("IoU: {:.4f}".format(iou))
  print("F1 Score: {:.4f}".format(f1))
  print()

print("Precision: {:.4f}".format(np.mean(precision_list)))
print("Recall: {:.4f}".format(np.mean(recall_list)))
print("Accuracy: {:.4f}".format(np.mean(accuracy_list)))
print("IoU: {:.4f}".format(np.mean(iou_list)))
print("F1 Score: {:.4f}".format(np.mean(f1_list)))

A0201_ELAGIGILTV_MART-1_Cancer_binder
TP: 3
TN: 7323
FP: 485
FN: 154

Precision: 0.0061
Recall: 0.0191
Accuracy: 0.9198
IoU: 0.0047
F1 Score: 0.0093

A0201_GILGFVFTL_Flu-MP_Influenza_binder
TP: 1313
TN: 6116
FP: 26
FN: 510

Precision: 0.9806
Recall: 0.7202
Accuracy: 0.9327
IoU: 0.7101
F1 Score: 0.8305

A0301_KLGGALQAK_IE-1_CMV_binder
TP: 1535
TN: 4311
FP: 1040
FN: 1079

Precision: 0.5961
Recall: 0.5872
Accuracy: 0.7340
IoU: 0.4201
F1 Score: 0.5916

A1101_AVFDRKSDAK_EBNA-3B_EBV_binder
TP: 134
TN: 6067
FP: 1201
FN: 563

Precision: 0.1004
Recall: 0.1923
Accuracy: 0.7785
IoU: 0.0706
F1 Score: 0.1319

A1101_IVTDFSVIK_EBNA-3B_EBV_binder
TP: 2224
TN: 5287
FP: 4
FN: 450

Precision: 0.9982
Recall: 0.8317
Accuracy: 0.9430
IoU: 0.8305
F1 Score: 0.9074

Precision: 0.5363
Recall: 0.4701
Accuracy: 0.8616
IoU: 0.4072
F1 Score: 0.4941
